# Intro to asyncio - croutines

## yield from

In [1]:
def sub_gen():
    yield 1.1
    yield 1.2

In [2]:
def gen():
    yield 1
    for i in sub_gen():
        yield i
    yield 2

In [3]:
for x in gen():
    print(x)

1
1.1
1.2
2


Isto s pomočjo yield form:

In [4]:
def sub_gen():
    yield 1.1
    yield 1.2
    
def gen():
    yield 1
    yield from sub_gen()
    yield 2
    
for x in gen():
    print(x)

1
1.1
1.2
2


In [5]:
def sub_gen():
    yield 1.1
    yield 1.2
    return "Done!"
    
def gen():
    yield 1
    result = yield from sub_gen()
    print("<--", result)
    yield 2
    
for x in gen():
    print(x)

1
1.1
1.2
<-- Done!
2


## Example chain

In [6]:
from itertools import chain

In [8]:
list(chain("ABC", range(3), [6,7,8]))

['A', 'B', 'C', 0, 1, 2, 6, 7, 8]

In [11]:
def chain(*iterables):
    print(iterables)
    for it in iterables:
        for i in it:
            yield i

In [12]:
list(chain("ABC", range(3), [6,7,8]))

('ABC', range(0, 3), [6, 7, 8])


['A', 'B', 'C', 0, 1, 2, 6, 7, 8]

In [15]:
def chain(*iterables):
    print(iterables)
    for it in iterables:
        yield from it

In [16]:
list(chain("ABC", range(3), [6,7,8]))

('ABC', range(0, 3), [6, 7, 8])


['A', 'B', 'C', 0, 1, 2, 6, 7, 8]

## Classic coroutines

In [18]:
def simple_coroutine():
    print("start")
    y = yield
    print(f"data: {y}")

In [19]:
my_coro = simple_coroutine()

In [20]:
my_coro

<generator object simple_coroutine at 0x7f7cf31ba9e0>

In [21]:
next(my_coro)

start


In [22]:
my_coro.send(42)

data: 42


StopIteration: 

In [23]:
def simple_coroutine(a):
    print("start a =", a)
    b = yield a
    print(f"rec: b={b}")
    c = yield a + b
    print(f"rec: c={c}")

In [24]:
my_coro2 = simple_coroutine(14)

In [25]:
from inspect import getgeneratorstate

In [26]:
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [27]:
next(my_coro2)

start a = 14


14

In [28]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [29]:
my_coro2.send(28)

rec: b=28


42

In [30]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [31]:
my_coro2.send(99)

rec: c=99


StopIteration: 

In [32]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

### Running average

In [33]:
def povprecje():
    total = 0.0
    count = 0
    avg = 0.0
    while True:
        term = yield avg
        total += term
        count += 1
        avg = total/count

In [34]:
coro_avg = povprecje()

In [35]:
next(coro_avg)

0.0

In [36]:
coro_avg.send(10)

10.0

In [37]:
coro_avg.send(30)

20.0

In [38]:
coro_avg.send(5)

15.0

In [39]:
coro_avg.close()

In [40]:
coro_avg.close()

In [41]:
coro_avg.send(5)

StopIteration: 

### Vračanje vrednosti iz corotiun

In [42]:
from typing import NamedTuple

class Result(NamedTuple):
    count: int
    average: float
        
class Sentinel:
    def __repr__(self):
        return f"<Sentinel>"
    
STOP = Sentinel()

def povprecje():
    total = 0.0
    count = 0
    avg = 0.0
    while True:
        term = yield
        print("rec: ", term)
        if isinstance(term, Sentinel):
            break
        total += term
        count += 1
        avg = total/count
    return Result(count, avg)

In [43]:
coro_avg = povprecje()

In [44]:
next(coro_avg)

In [45]:
coro_avg.send(10)

rec:  10


In [46]:
coro_avg.send(30)

rec:  30


In [47]:
coro_avg.send(20)

rec:  20


In [48]:
coro_avg.send(10)

rec:  10


In [49]:
coro_avg.close()

In [50]:
coro_avg = povprecje()
next(coro_avg)
coro_avg.send(30)
coro_avg.send(20)
coro_avg.send(10)

try:
    coro_avg.send(STOP)
except StopIteration as exc:
    result = exc.value

print(result)

rec:  30
rec:  20
rec:  10
rec:  <Sentinel>
Result(count=3, average=20.0)


In [51]:
def compute():
    res = yield from povprecje()
    print(f"res = {res}")
    return res

In [52]:
com = compute()

In [53]:
for v in [None, 10, 20, 30, STOP]:
    try:
        com.send(v)
    except StopIteration as exc:
        result = exc.value

print(result)

rec:  10
rec:  20
rec:  30
rec:  <Sentinel>
res = Result(count=3, average=20.0)
Result(count=3, average=20.0)
